The program is designed for finding duplicate songs in the Million Song Dataset using LSH algorithm with cosine similarity.

It consists of the controler used for user interface  and of all functions which are needed for LSH algorithm. The user can decide about:
- the number of songs that will be analysed
- the subset of features
- the number of bands and rows that will be used
- the maximum angle or distance for cosine similarity.

Pairs of duplicate songs (or the message that they do not exist) is the output.

Additionaly, user can change the subset of features afterwards. Also it is possible to compare the runtime of LSH algorithm and naive algorithm that compares all possible pairs.

In [1]:
#IMPORTS AND HELPER METHODS
import gzip
import tarfile
import math
import numpy as np
import random
import pandas as pd
from sklearn import preprocessing
import collections
import time

def stringToInt(s):
    try:
        return int(s)
    except ValueError:
        print('String to int convertion error')
        return False

In [2]:
#OPEN FILE
tar = tarfile.open(r'millionsongsubset_full.tar.gz', 'r')
members = tar.getmembers()

tar.extract(members[5])
summary = pd.HDFStore(members[5].name)

In [3]:
#USER INPUT: SET SUBSET SIZE
def setSubsetSize():
    while True:
        numOfSongs = input('How large should the subset be?\n')
        numOfSongs = stringToInt(numOfSongs)
        if numOfSongs > 0:
            break
        print('Please select a number over 0')
    return numOfSongs

#USER INPUT: SELECT FEATURES
def selectFeatures():
    print ('Select which features you want to consider (seperated by comma). For example: 1, 3, 8\n\n1. Duration\n2. End of fade in\n3. Key\n4. Key confidence\n5. Loudness\n6. Mode\n7. Mode confidence\n8. Start of fade out\n9. Tempo\n10. Time signature\n11. Time signature confidence\n')
    while True:
        userInput = input('My features: ')
        features = [stringToInt(x.strip()) for x in userInput.split(',')]
        if all(f>0 and f<12 for f in features):
            break
        print('Features must be in range (1,11)')
    return features

#USER INPUT: SET PARAMETERS
def setParameters():
    param = input('Parameters that will be used are: bands = 3, rows = 64, eps = 2. Do you want to change it? y/n')
        
    if param == 'y':
        while True:
            b = stringToInt(input('How many bands should be used? b = '))
            if b != False and b > 0:
                break
            print ('Number of bands must be positive')
            
        while True:
            r = stringToInt(input('How many rows should be used? r = '))
            if r != False and r > 0:
                break
            print ('Number of rows must be positive')
        
        while True:
            eps_answer = input('Do you want to use angle or distance? a/d ')
            if check_input_string(eps_answer, ['a', 'd']):
                if eps_answer == 'a':
                    eps = float(input('Please insert the angle in degrees: angle = '))
                    eps_type = True
                else:
                    eps = float(input('Please insert the cosine distance: distance = '))
                    eps_type = False
                break
        
        return (b, r, eps, eps_type)
    return (3, 64, 2, True)

#Check user input
def check_input_string(user_input, correct_values):
    for value in correct_values:
        if value == user_input:
            return True
    print ('We are sorry, but input is irregular. Please be careful next time.')
    return False



In [4]:
#READ IN FEATURES, STANDARDIZE AND INPUT INTO NUMPY ARRAY
def readFeatures(features, numOfSongs):
    songs = np.empty([len(features)+1, numOfSongs], dtype=float)
    
    track_ids = summary['/analysis/songs'].track_id
    songids= np.array(track_ids[0:numOfSongs])
    
    count = 0
    for x in features:
        if x == 1:
            duration = summary['/analysis/songs'].duration
            durationN = preprocessing.scale(duration)
            songs[count] = np.array(durationN[0:numOfSongs])
            count+=1
        elif x == 2:
            endFadeIn = summary['/analysis/songs'].end_of_fade_in
            endFadeInN = preprocessing.scale(endFadeIn)
            songs[count] = np.array(endFadeInN[0:numOfSongs])
            count+=1
        elif x == 3:
            key = summary['/analysis/songs'].key
            keyN = preprocessing.scale(key)
            songs[count] = np.array(keyN[0:numOfSongs])
        elif x == 4:
            keyConfidence = summary['/analysis/songs'].key_confidence
            keyConfidenceN = preprocessing.scale(keyConfidence)
            songs[count] = np.array(keyConfidenceN[0:numOfSongs])
            count+=1
        elif x == 5:
            loudness = summary['/analysis/songs'].loudness
            loudnessN = preprocessing.scale(loudness)
            songs[count] = np.array(loudnessN[0:numOfSongs])
            count+=1
        elif x == 6:
            mode = summary['/analysis/songs']['mode']
            modeN = preprocessing.scale(mode)
            songs[count] = np.array(modeN[0:numOfSongs])
            count+=1
        elif x == 7:
            modeConfidence = summary['/analysis/songs'].mode_confidence
            modeConfidenceN = preprocessing.scale(modeConfidence)
            songs[count] = np.array(modeConfidenceN[0:numOfSongs])
            count+=1
        elif x == 8:
            startFadeOut = summary['/analysis/songs'].start_of_fade_out
            startFadeOutN = preprocessing.scale(startFadeOut)
            songs[count] = np.array(durationN[0:numOfSongs])
            count+=1
        elif x == 9:
            tempo = summary['/analysis/songs'].tempo
            tempoN = preprocessing.scale(tempo)
            songs[count] = np.array(tempoN[0:numOfSongs])
            count+=1
        elif x == 10:
            timeSignature = summary['/analysis/songs'].time_signature
            timeSignatureN = preprocessing.scale(timeSignature)
            songs[count] = np.array(timeSignatureN[0:numOfSongs])
            count+=1
        elif x == 11:
            timeSignatureConfidence = summary['/analysis/songs'].time_signature_confidence
            timeSignatureConfidenceN = preprocessing.scale(timeSignatureConfidence)
            songs[count] = np.array(timeSignatureConfidenceN[0:numOfSongs])
            count+=1
    return (songs, songids)

In [5]:
# cosine_distance calculates cosine distance
def cosine_distance(x,y):
    numerator = np.dot(x,y)
    denominator = math.sqrt(np.dot(x,x)*np.dot(y,y))
    return round(numerator/float(denominator),3)

# hash_func creates bools for input vector and projection vectors and converts them to integer
def hash_func(vec, projections):
    bools = np.dot(projections, vec) > 0
    #convert to int:
    p = 0
    for i,j in enumerate(bools):
        if j: p+= 1<<i
    return p

#find_similar_songs uses LSH to find candidates and then using the other function returns real duplicates
def find_similar_songs(songs, ids, num_bands = 3, num_rows = 64, eps = 2, eps_type = True):
    songs = songs.transpose()
    
    #convert distance to angle
    if eps_type == False:
        eps = math.acos(1-eps)
        eps = eps * 180 / math.pi #rad to degrees
    
    #dimensions of songs: num_songs X num_features
    num_features = len(songs[0])
    candidates_pairs = set()
    
    for band in range(num_bands):
        #for each band generate random vectors 
        projections = np.random.randn(num_rows, num_features)
        table = collections.defaultdict(list)
        
        for i in range(len(songs)):
            hp = hash_func(songs[i], projections) #for each song find the bucket to which belongs
            table[str(hp)].append(i) #add songs to corresponding bucket
      
        
        for c in table:
            if len(c) > 1: 
                #if in a bucket is more than one song generate candidate pairs (song, song)
                for c1 in range(len(table[c])-1):
                    for c2 in range(c1+1, len(table[c])):
                        candidates_pairs.add((table[c][c1], table[c][c2]))
                        
    #for all potential duplicates (candidates) check if they are real duplicates                    
    duplicates = find_duplicates(candidates_pairs, songs, ids, eps)
    #return set of pairs (songId, songID)
    return duplicates


#find_duplicates calculates cosine distance for each pair and if the angle between them is less then input angle they are duplicates
#returns all duplicates 
def find_duplicates(pairs, songs, ids, eps):
    
    duplicates = set()
    
    for pair in pairs:
        x = songs[pair[0]]
        y = songs[pair[1]]
        #calculate the angle as acos of the cosine distance and convert it to degrees
        angle = (math.acos(cosine_distance(x,y))*180)/math.pi
        if angle < eps: 
            duplicates.add((ids[pair[0]], ids[pair[1]]))
    #return set of pairs (songId, songID)
    return duplicates
            

#find_duplicate_naive compares all songs and returns duplicates in respect to cosine distance
def find_duplicates_naive(songs, ids, eps):
    songs = songs.transpose()
    duplicates = set()
    
    for i in range(len(songs)-1):
        for j in range(i+1, len(songs)):
            x = songs[i]
            y = songs[j]
            angle = (math.acos(cosine_distance(x,y))*180)/math.pi
            if angle < eps:
                duplicates.add((ids[i], ids[j]))
    return duplicates

#print_duplicates prints all pairs of similar songs
def print_duplicates(duplicates):
    print ('-------------------------------------')
    if len(duplicates) == 0:
        print ('\n 0 duplicates in this dataset')
    else:
        print ('\n Duplicates are:')
        count = 1
        for duplicate in duplicates:
            print (count, duplicate)
            count = count + 1

In [16]:
#MAIN CONTROLLER
run = True

print ('Welcome to the duplicate song search')
print ('Please select your desired subset size and feature set')

while run:
    #Set parameters
    numOfSongs = setSubsetSize()
    features = selectFeatures()
    
    (songs, ids) = readFeatures(features, numOfSongs)
    print ('Chosen features: ', features)
    (b, r, eps, eps_type) = setParameters()
    songs
        
    start_time = time.time()
    #Run duplication search definition
    duplicates_lsh = find_similar_songs(songs, ids, b, r, eps, eps_type)
    time_lsh = time.time() - start_time
    print_duplicates(duplicates_lsh)
    print('\nTime taken to compute: ', round(time_lsh*1000, 2), 'ms')
    
    #Compare running times
    comp_answer = input('\nDo you want to compare running time with the naive implementation? y/n ')
    if comp_answer == 'y':
        print('\n Please wait')
        start_time = time.time()
        duplicates_naive = find_duplicates_naive(songs, ids, eps)
        time_naive = time.time() - start_time
        print('LSH algorithm:')
        print('Time:', round(time_lsh*1000, 3), 'ms',  'Number of duplicates:', len(duplicates_lsh))
        print('Naive algorithm:')
        print('Time:', round(time_naive*1000, 3), 'ms', 'Number of duplicates:', len(duplicates_naive))
    
    #Add/remove features
    change_features = True
    while change_features:
        answer_feat = input('Do you want to change features? y/n ')
        if answer_feat == 'y':
            features = selectFeatures()
            readFeatures(features, numOfSongs)
            print ('Chosen features: ', features)
            (songs, ids) = readFeatures(features, numOfSongs)
            duplicates = find_similar_songs(songs, ids, b, r, eps, eps_type)
            print_duplicates(duplicates)
        else:
            change_features = False
    
    answer = input('Do you wish to try another search? y/n ')
    if answer == 'n':
        run = False 
        print ('--------------------------------')
        print ('Thank you for using our program!')

   

Welcome to the duplicate song search
Please select your desired subset size and feature set
How large should the subset be?
100
Select which features you want to consider (seperated by comma). For example: 1, 3, 8

1. Duration
2. End of fade in
3. Key
4. Key confidence
5. Loudness
6. Mode
7. Mode confidence
8. Start of fade out
9. Tempo
10. Time signature
11. Time signature confidence

My features: 1,2
Chosen features:  [1, 2]
Parameters that will be used are: bands = 3, rows = 64, eps = 2. Do you want to change it? y/n
-------------------------------------

 Duplicates are:
1 ('TRACCLY12903CDD7C4', 'TRACMXG12903CA2541')
2 ('TRACJJZ12903CCBCDA', 'TRACMMG128F146D705')
3 ('TRACCSW128F148C7C3', 'TRACOLN12903CCAF8B')
4 ('TRACIJF128F424317B', 'TRACHYC12903CEAAB6')
5 ('TRACOHQ128F424C5EF', 'TRACMJP128F9349362')
6 ('TRACXGV128F9349C63', 'TRACMTN128F92FB689')
7 ('TRACCXJ128F92E984B', 'TRACCLY12903CDD7C4')
8 ('TRACHAL128F427DD25', 'TRACHSH12903CEBC2C')
9 ('TRACCXJ128F92E984B', 'TRACXRL128F9317E

In [22]:
duplicates

{('TRACBWX128F92FD725', 'TRAIDIU128F92F11C7'),
 ('TRACCKS128F42B77AE', 'TRAJOPN128F423BEC3'),
 ('TRACFOV128F9340D81', 'TRAIATD128F9331CD6'),
 ('TRACLHC128F9343C39', 'TRAIINI128F426C8DD'),
 ('TRACOAF128F145D996', 'TRAIXQL128F1467D66'),
 ('TRACUEE128F427EA05', 'TRAJFDU128F426A5F4'),
 ('TRACWRW12903CC3CA2', 'TRAJFMH12903D052DC'),
 ('TRACWVL128F93016B4', 'TRACQDO12903CAE2DA'),
 ('TRAIACZ128F428C60A', 'TRAIKRJ128F4275BED'),
 ('TRAIBHP128F149C2ED', 'TRAIFJE128F933046A'),
 ('TRAIHFF128EF351817', 'TRAHISQ128F4243C7B'),
 ('TRAIQVJ128F93235B3', 'TRAHOOP128F42B62AB'),
 ('TRAJAPH128F4271357', 'TRAILIV128F932C85E'),
 ('TRAJARM128F426379B', 'TRAICEE12903D04035'),
 ('TRAJHMX12903CD42C1', 'TRAIVMM12903CFAD62'),
 ('TRAJWGJ128F42ADC43', 'TRAJTCL128F1460CDA')}